In [1]:

import numpy as np 
import pandas as pd 
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from scipy.interpolate import UnivariateSpline
from sklearn import linear_model
import xgboost as xgb
import numpy as np
import pandas as pd


from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test1=pd.read_csv('test.csv')

In [4]:
train

,id,date,speed
0,0,1/1/2017 0:00,43.002930
1,1,1/1/2017 1:00,46.118696
2,2,1/1/2017 2:00,44.294158
3,3,1/1/2017 3:00,41.067468
4,4,1/1/2017 4:00,46.448653
...,...,...,...
14001,14001,31/12/2018 12:00,19.865269
14002,14002,31/12/2018 15:00,17.820375
14003,14003,31/12/2018 16:00,12.501851
14004,14004,31/12/2018 18:00,15.979319


In [132]:
test

,id,date,datetime,year,month,day,time,weekend,holidy,timesplit,data_ymd
0,0,1/1/2018 2:00,2018-01-01 02:00:00,2018,1,1,2,0,1,Late_Night,2018-01-01
1,1,1/1/2018 5:00,2018-01-01 05:00:00,2018,1,1,5,0,1,Early_Morning,2018-01-01
2,2,1/1/2018 7:00,2018-01-01 07:00:00,2018,1,1,7,0,1,Early_Morning,2018-01-01
3,3,1/1/2018 8:00,2018-01-01 08:00:00,2018,1,1,8,0,1,Morning,2018-01-01
4,4,1/1/2018 10:00,2018-01-01 10:00:00,2018,1,1,10,0,1,Morning,2018-01-01
...,...,...,...,...,...,...,...,...,...,...,...
3499,3499,31/12/2018 17:00,2018-12-31 17:00:00,2018,12,31,17,0,0,Evening,2018-12-31
3500,3500,31/12/2018 19:00,2018-12-31 19:00:00,2018,12,31,19,0,0,Evening,2018-12-31
3501,3501,31/12/2018 21:00,2018-12-31 21:00:00,2018,12,31,21,0,0,Night,2018-12-31
3502,3502,31/12/2018 22:00,2018-12-31 22:00:00,2018,12,31,22,0,0,Night,2018-12-31


In [133]:
train['datetime']=train['date'].apply(lambda x:datetime.datetime(  int(x.split()[0].split('/')[2]),int(x.split()[0].split('/')[1]),int(x.split()[0].split('/')[0]),int(x.split()[1].split(':')[0])) )

In [134]:
test['datetime']=test['date'].apply(lambda x:datetime.datetime(  int(x.split()[0].split('/')[2]),int(x.split()[0].split('/')[1]),int(x.split()[0].split('/')[0]),int(x.split()[1].split(':')[0])) )

In [135]:
# date_range=pd.date_range("2017-01-01 00:00:00","2018-12-31 23:00:00",freq='1H')
# date_range[:5]

In [136]:
# new_index=pd.MultiIndex.from_product([date_range],names=['datetime'])
# new_df=pd.DataFrame(index=new_index).reset_index()
# new_df.head()

In [137]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14006 entries, 0 to 14005
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         14006 non-null  int64         
 1   date       14006 non-null  object        
 2   speed      14006 non-null  float64       
 3   datetime   14006 non-null  datetime64[ns]
 4   year       14006 non-null  int64         
 5   month      14006 non-null  int64         
 6   day        14006 non-null  int64         
 7   time       14006 non-null  int64         
 8   weekend    14006 non-null  int64         
 9   holidy     14006 non-null  int64         
 10  timesplit  14006 non-null  object        
 11  data_ymd   14006 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(7), object(3)
memory usage: 1.3+ MB


In [138]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         3504 non-null   int64         
 1   date       3504 non-null   object        
 2   datetime   3504 non-null   datetime64[ns]
 3   year       3504 non-null   int64         
 4   month      3504 non-null   int64         
 5   day        3504 non-null   int64         
 6   time       3504 non-null   int64         
 7   weekend    3504 non-null   int64         
 8   holidy     3504 non-null   int64         
 9   timesplit  3504 non-null   object        
 10  data_ymd   3504 non-null   object        
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 301.2+ KB


In [139]:
train.drop('id',axis=1)
test.drop('id',axis=1)

,date,datetime,year,month,day,time,weekend,holidy,timesplit,data_ymd
0,1/1/2018 2:00,2018-01-01 02:00:00,2018,1,1,2,0,1,Late_Night,2018-01-01
1,1/1/2018 5:00,2018-01-01 05:00:00,2018,1,1,5,0,1,Early_Morning,2018-01-01
2,1/1/2018 7:00,2018-01-01 07:00:00,2018,1,1,7,0,1,Early_Morning,2018-01-01
3,1/1/2018 8:00,2018-01-01 08:00:00,2018,1,1,8,0,1,Morning,2018-01-01
4,1/1/2018 10:00,2018-01-01 10:00:00,2018,1,1,10,0,1,Morning,2018-01-01
...,...,...,...,...,...,...,...,...,...,...
3499,31/12/2018 17:00,2018-12-31 17:00:00,2018,12,31,17,0,0,Evening,2018-12-31
3500,31/12/2018 19:00,2018-12-31 19:00:00,2018,12,31,19,0,0,Evening,2018-12-31
3501,31/12/2018 21:00,2018-12-31 21:00:00,2018,12,31,21,0,0,Night,2018-12-31
3502,31/12/2018 22:00,2018-12-31 22:00:00,2018,12,31,22,0,0,Night,2018-12-31


In [140]:
# train['datetime']=pd.to_datetime(train['date'].map(lambda x: x[:]))
# # train.head()

In [12]:
train['year']=train['date'].apply(lambda y:int(y.split()[0].split('/')[2]))
test['year']=test['date'].apply(lambda y:int(y.split()[0].split('/')[2]))

In [13]:
train['month']=train['date'].apply(lambda m:int(m.split()[0].split('/')[1]))
test['month']=test['date'].apply(lambda m:int(m.split()[0].split('/')[1]))

In [14]:
train['day']=train['date'].apply(lambda d:int(d.split()[0].split('/')[0]))
test['day']=test['date'].apply(lambda d:int(d.split()[0].split('/')[0]))

In [15]:
train['time']=train['date'].apply(lambda t:int(t.split()[1].split(':')[0]))
test['time']=test['date'].apply(lambda t:int(t.split()[1].split(':')[0]))

In [16]:
# train['date_year']=train.datetime.map(lambda x: x.strftime('%Y')).astype(int)
# train['date_month']=train.datetime.map(lambda x: x.strftime('%m')).astype(int)
# train['date_day']=train.datetime.map(lambda x: x.strftime('%d')).astype(int)
# train['date_ymd']=train.datetime.map(lambda x: x.strftime('%Y-%m-%d'))
# train['date_hour']=train.datetime.map(lambda x: x.strftime('%H')).astype(int)
# train

In [17]:
train.head(30)

,id,date,speed,datetime,year,month,day,time
0,0,1/1/2017 0:00,43.002930,2017-01-01 00:00:00,2017,1,1,0
1,1,1/1/2017 1:00,46.118696,2017-01-01 01:00:00,2017,1,1,1
2,2,1/1/2017 2:00,44.294158,2017-01-01 02:00:00,2017,1,1,2
3,3,1/1/2017 3:00,41.067468,2017-01-01 03:00:00,2017,1,1,3
4,4,1/1/2017 4:00,46.448653,2017-01-01 04:00:00,2017,1,1,4
5,5,1/1/2017 5:00,46.797766,2017-01-01 05:00:00,2017,1,1,5
6,6,1/1/2017 6:00,44.404925,2017-01-01 06:00:00,2017,1,1,6
7,7,1/1/2017 7:00,45.255897,2017-01-01 07:00:00,2017,1,1,7
8,8,1/1/2017 8:00,45.680859,2017-01-01 08:00:00,2017,1,1,8
9,9,1/1/2017 9:00,48.435676,2017-01-01 09:00:00,2017,1,1,9


In [18]:
# trainset.to_csv('ttt.csv')

In [142]:
# Whether a day is weekend or not
train['weekend']=train['datetime'].apply(lambda x:0 if x.weekday() in [0,1,2,3,4] else 1)
test['weekend']=test['datetime'].apply(lambda x:0 if x.weekday() in [0,1,2,3,4] else 1)

In [20]:
# train['dd']=train['date_month'].map(str)+train['date_day'].map(str)

In [21]:
# train['d']=train['date_month'].map(str)+train['date_day'].map(str)+train['date_hour'].map(str)

In [22]:
## train['ddd']=train['date_year'].map(str)+train['date_month'].map(str)+train['date_day'].map(str)

In [23]:
# train['dddd']=train['date_year'].map(str)+train['date_month'].map(str)+train['date_day'].map(str)+train['date_hour'].map(str)

###   we can obtain the holiday information on the website https://www.gov.hk/en/about/abouthk/holiday/

In [24]:
# Whether a day is Hong Kong General Holidays
train['holidy']=train['datetime'].apply(lambda x:1 if (x.date().strftime('%Y-%m-%d') in [
           '2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14','2017-04-15','2017-04-17','2017-05-01','2017-05-03','2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26',
           '2018-01-01','2018-02-16','2018-02-17','2018-02-19','2018-03-30','2018-03-31','2018-04-02','2018-04-05','2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17','2018-12-25','2018-12-26'])
           or(x.weekday() in[6]) else 0)
test['holidy']=test['datetime'].apply(lambda x:1 if (x.date().strftime('%Y-%m-%d') in [
           '2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14','2017-04-15','2017-04-17','2017-05-01','2017-05-03','2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26',
           '2018-01-01','2018-02-16','2018-02-17','2018-02-19','2018-03-30','2018-03-31','2018-04-02','2018-04-05','2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17','2018-12-25','2018-12-26'])
           or(x.weekday() in[6]) else 0)

In [25]:
# if ((prods['hour'] < 4) & (prods['hour'] > 8 )):
#     prods['session'] = 'Early Morning'
# elif ((prods['hour'] < 8) & (prods['hour'] > 12 )):
#     prods['session'] = 'Morning'
# elif ((prods['hour'] < 12) & (prods['hour'] > 16 )):
#     prods['session'] = 'Noon'
# elif ((prods['hour'] < 16) & (prods['hour'] > 20 )):
#     prods['session'] = 'Eve'
# elif ((prods['hour'] < 20) & (prods['hour'] > 24 )):
#     prods['session'] = 'Night'
# elif ((prods['hour'] < 24) & (prods['hour'] > 4 )):
#     prods['session'] = 'Late Night'

In [143]:

#     df.loc[df['date'].isin(
#         ['2017-04-02', '2017-04-03', '2017-04-04', '2017-04-29', '2017-04-30', '2017-05-01',
#          '2017-05-28', '2017-05-29', '2017-05-30']), 'vacation'] = 1

#     df.loc[~df['date'].isin(
#         ['2017-04-02', '2017-04-03', '2017-04-04', '2017-04-29', '2017-04-30', '2017-05-01',
#          '2017-05-28', '2017-05-29', '2017-05-30']), 'vacation'] = 0


In [85]:
def timesplit(x):
    Early_Morning = [4,5,6,7]
    Morning = [8,9,10,11]
    Afternoon = [12,13,14,15]
    Evening = [16,17,18,19]
    Night = [20,21,22,23]
    Late_Night = [0,1,2,3]
    if x in Early_Morning:
        return 'Early_Morning'
    elif x in Morning:
        return 'Morning'
    elif x in Afternoon:
        return 'Afternoon'
    elif x in Evening:
        return 'Evening'
    elif x in Night:
        return 'Night'
    else:
        return 'Late_Night'
    
train['timesplit']=train['time'].apply(timesplit)
test['timesplit']=test['time'].apply(timesplit)
train.head()

,id,date,speed,datetime,year,month,day,time,weekend,holidy,timesplit,data_ymd
0,0,1/1/2017 0:00,43.002930,2017-01-01 00:00:00,2017,1,1,0,1,1,Late_Night,2017-01-01
1,1,1/1/2017 1:00,46.118696,2017-01-01 01:00:00,2017,1,1,1,1,1,Late_Night,2017-01-01
2,2,1/1/2017 2:00,44.294158,2017-01-01 02:00:00,2017,1,1,2,1,1,Late_Night,2017-01-01
3,3,1/1/2017 3:00,41.067468,2017-01-01 03:00:00,2017,1,1,3,1,1,Late_Night,2017-01-01
4,4,1/1/2017 4:00,46.448653,2017-01-01 04:00:00,2017,1,1,4,1,1,Early_Morning,2017-01-01


In [141]:

#     df['minute'] = df['time_interval_begin'].dt.minute
#     df['hour'] = df['time_interval_begin'].dt.hour
#     df['day'] = df['time_interval_begin'].dt.day
#     df['week_day'] = df['time_interval_begin'].map(lambda x: x.weekday() + 1)
#     df['month'] = df['time_interval_begin'].dt.month


In [144]:
weather=pd.read_csv("weather.csv")
weather.head()

,Unnamed: 0,year,month,day,MPressure(hPa),MaxTemp(℃),MinTemp(℃),MCloud(%),TRainfall(mm),#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
0,0,2017,1,1,1021.7,20.8,18.4,72,-,0,60,34.2
1,1,2017,1,2,1020.2,23.3,18.4,28,-,0,70,17.7
2,2,2017,1,3,1019.8,21.3,18.9,56,-,5,70,26.1
3,3,2017,1,4,1018.7,21.7,18.7,51,-,0,70,27.7
4,4,2017,1,5,1016.9,23.4,18.9,61,-,0,40,14.3


In [145]:
weather.drop("Unnamed: 0",axis=1,inplace=True)

In [180]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 730 non-null    int64  
 1   month                730 non-null    int64  
 2   day                  730 non-null    int64  
 3   MPressure(hPa)       730 non-null    float64
 4   MaxTemp(℃)           730 non-null    float64
 5   MinTemp(℃)           730 non-null    float64
 6   MCloud(%)            730 non-null    int64  
 7   TRainfall(mm)        730 non-null    object 
 8   #hRedVisi(h)         730 non-null    int64  
 9   WindDirect(degrees)  730 non-null    int64  
 10  MWindSpeed(km/h)     730 non-null    float64
 11  cleaned              730 non-null    object 
 12  data_ymd             730 non-null    object 
dtypes: float64(4), int64(6), object(3)
memory usage: 74.3+ KB


In [181]:
weather['year'].apply(lambda t:int(t))
weather['month'].apply(lambda t:int(t))
weather['day'].apply(lambda t:int(t))

0       1
1       2
2       3
3       4
4       5
       ..
725    27
726    28
727    29
728    30
729    31
Name: day, Length: 730, dtype: int64

In [182]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 730 non-null    int64  
 1   month                730 non-null    int64  
 2   day                  730 non-null    int64  
 3   MPressure(hPa)       730 non-null    float64
 4   MaxTemp(℃)           730 non-null    float64
 5   MinTemp(℃)           730 non-null    float64
 6   MCloud(%)            730 non-null    int64  
 7   TRainfall(mm)        730 non-null    object 
 8   #hRedVisi(h)         730 non-null    int64  
 9   WindDirect(degrees)  730 non-null    int64  
 10  MWindSpeed(km/h)     730 non-null    float64
 11  cleaned              730 non-null    object 
 12  data_ymd             730 non-null    object 
dtypes: float64(4), int64(6), object(3)
memory usage: 74.3+ KB


In [183]:
weather.drop(['year','month','day'],axis=1).describe()

,MPressure(hPa),MaxTemp(℃),MinTemp(℃),MCloud(%),#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,1012.883562,26.564110,22.012877,70.315068,1.032877,140.671233,23.796849
std,6.504514,5.354513,5.088792,22.101572,2.924569,110.679229,10.542179
min,990.900000,10.600000,6.800000,4.000000,0.000000,10.000000,4.000000
25%,1008.400000,22.700000,17.900000,59.000000,0.000000,60.000000,15.900000
50%,1013.400000,27.400000,23.000000,79.000000,0.000000,80.000000,23.600000
75%,1017.500000,31.100000,26.400000,88.000000,0.000000,220.000000,30.650000
max,1028.200000,36.600000,29.800000,100.000000,21.000000,360.000000,101.300000


In [150]:
weather.isna().count()

year                   730
month                  730
day                    730
MPressure(hPa)         730
MaxTemp(℃)             730
MinTemp(℃)             730
MCloud(%)              730
TRainfall(mm)          730
#hRedVisi(h)           730
WindDirect(degrees)    730
MWindSpeed(km/h)       730
dtype: int64

In [175]:
weather['cleaned']=weather['TRainfall(mm)'].apply(lambda x:0 if x in ['-','Trace'] else x)

In [176]:
train['data_ymd']=train['datetime'].apply(lambda x:x.date())
def addup(x):
    return datetime.datetime(x[0],x[1],x[2]).date()
weather['data_ymd']=weather.apply(addup,axis=1)

In [177]:
trainle=train[['datetime','year','month','day','time','speed','weekend','holidy','timesplit','data_ymd']]

In [178]:
weatherri=weather[['MaxTemp(℃)','MinTemp(℃)','MCloud(%)','cleaned','#hRedVisi(h)','WindDirect(degrees)','MWindSpeed(km/h)','data_ymd']]

In [179]:
trained=pd.merge(trainle,weatherri,on='data_ymd',how='left')

In [156]:
trained=trained.drop(['datetime','data_ymd'],axis=1)

trained.head()

,year,month,day,time,speed,weekend,holidy,timesplit,MaxTemp(℃),MinTemp(℃),MCloud(%),cleaned,#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
0,2017,1,1,0,43.002930,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
1,2017,1,1,1,46.118696,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
2,2017,1,1,2,44.294158,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
3,2017,1,1,3,41.067468,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
4,2017,1,1,4,46.448653,1,1,Early_Morning,20.8,18.4,72,0,0,60,34.2


In [157]:
trained.columns

Index(['year', 'month', 'day', 'time', 'speed', 'weekend', 'holidy',
       'timesplit', 'MaxTemp(℃)', 'MinTemp(℃)', 'MCloud(%)', 'cleaned',
       '#hRedVisi(h)', 'WindDirect(degrees)', 'MWindSpeed(km/h)'],
      dtype='object')

In [158]:
trained.head(30)

,year,month,day,time,speed,weekend,holidy,timesplit,MaxTemp(℃),MinTemp(℃),MCloud(%),cleaned,#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
0,2017,1,1,0,43.002930,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
1,2017,1,1,1,46.118696,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
2,2017,1,1,2,44.294158,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
3,2017,1,1,3,41.067468,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
4,2017,1,1,4,46.448653,1,1,Early_Morning,20.8,18.4,72,0,0,60,34.2
5,2017,1,1,5,46.797766,1,1,Early_Morning,20.8,18.4,72,0,0,60,34.2
6,2017,1,1,6,44.404925,1,1,Early_Morning,20.8,18.4,72,0,0,60,34.2
7,2017,1,1,7,45.255897,1,1,Early_Morning,20.8,18.4,72,0,0,60,34.2
8,2017,1,1,8,45.680859,1,1,Morning,20.8,18.4,72,0,0,60,34.2
9,2017,1,1,9,48.435676,1,1,Morning,20.8,18.4,72,0,0,60,34.2


In [159]:
trained.to_csv('trained.csv',header=True, index=None,sep=',',mode='w')

In [160]:
trained.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14006 entries, 0 to 14005
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 14006 non-null  int64  
 1   month                14006 non-null  int64  
 2   day                  14006 non-null  int64  
 3   time                 14006 non-null  int64  
 4   speed                14006 non-null  float64
 5   weekend              14006 non-null  int64  
 6   holidy               14006 non-null  int64  
 7   timesplit            14006 non-null  object 
 8   MaxTemp(℃)           14006 non-null  float64
 9   MinTemp(℃)           14006 non-null  float64
 10  MCloud(%)            14006 non-null  int64  
 11  cleaned              14006 non-null  object 
 12  #hRedVisi(h)         14006 non-null  int64  
 13  WindDirect(degrees)  14006 non-null  int64  
 14  MWindSpeed(km/h)     14006 non-null  float64
dtypes: float64(4), int64(9), object(2)
m

In [161]:
trained.describe()

,year,month,day,time,speed,weekend,holidy,MaxTemp(℃),MinTemp(℃),MCloud(%),#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
count,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000,14006.000000
mean,2017.375268,6.491932,15.679138,11.529202,32.779118,0.289019,0.190990,26.567078,22.043217,69.986363,1.049479,137.534628,23.929487
std,0.484209,3.435815,8.793388,6.926150,13.573813,0.453323,0.393095,5.330305,5.064723,22.010725,2.947944,108.438981,10.492915
min,2017.000000,1.000000,1.000000,0.000000,2.573417,0.000000,0.000000,10.600000,6.800000,4.000000,0.000000,10.000000,4.000000
25%,2017.000000,4.000000,8.000000,6.000000,19.301089,0.000000,0.000000,22.400000,17.800000,58.000000,0.000000,60.000000,15.900000
50%,2017.000000,6.000000,16.000000,12.000000,36.580595,0.000000,0.000000,27.400000,22.900000,78.000000,0.000000,80.000000,23.700000
75%,2018.000000,9.000000,23.000000,18.000000,45.877665,1.000000,0.000000,31.200000,26.500000,87.000000,0.000000,220.000000,30.800000
max,2018.000000,12.000000,31.000000,23.000000,53.161286,1.000000,1.000000,36.600000,29.800000,100.000000,21.000000,360.000000,101.300000


In [162]:
# train=train.drop("date_ymd",axis=1)

In [163]:
trained

,year,month,day,time,speed,weekend,holidy,timesplit,MaxTemp(℃),MinTemp(℃),MCloud(%),cleaned,#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
0,2017,1,1,0,43.002930,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
1,2017,1,1,1,46.118696,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
2,2017,1,1,2,44.294158,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
3,2017,1,1,3,41.067468,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
4,2017,1,1,4,46.448653,1,1,Early_Morning,20.8,18.4,72,0,0,60,34.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14001,2018,12,31,12,19.865269,0,0,Afternoon,15.6,11.8,77,0,0,360,26.8
14002,2018,12,31,15,17.820375,0,0,Afternoon,15.6,11.8,77,0,0,360,26.8
14003,2018,12,31,16,12.501851,0,0,Evening,15.6,11.8,77,0,0,360,26.8
14004,2018,12,31,18,15.979319,0,0,Evening,15.6,11.8,77,0,0,360,26.8


In [164]:
target=['speed']
cs=['weekend','holidy','timesplit']
ns=['year','month','day','time','MaxTemp(℃)','MinTemp(℃)','MCloud(%)',
          'cleaned','#hRedVisi(h)','WindDirect(degrees)',
          'MWindSpeed(km/h)']#'ddd','dd','ddd'

In [165]:
#  train=train.drop(['TRainfall(mm)_num','MWindSpeed(km/h)','#hRedVisi(h)','MaxTemp(℃)',
#           'MinTemp(℃)','MCloud(%)','WindDirect(degrees)'],axis=1)

In [166]:
# train=train.drop(['ddd','dddd','dd','d','MPressure(hPa)'],axis=1)
# test=test.drop(['ddd','dddd','dd','d','MPressure(hPa)'],axis=1)

In [167]:
# 'MCloud(%)',
# train=train.drop(['MCloud(%)'],axis=1)
# test=test.drop(['MCloud(%)'],axis=1)

In [168]:
# Creating pipeline to transform data
numeric_transformer=Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer=Pipeline(steps=[
    ('oneHot',OneHotEncoder())])

preprocessor=ColumnTransformer(transformers=[
    ('num',numeric_transformer,ns),
    ('cat',categorical_transformer,cs)])

data_transformed=preprocessor.fit_transform(trained)

In [169]:

X_train,X_test,y_train,y_test=train_test_split(data_transformed,trained['speed'],test_size=0.15,random_state=42)



In [170]:
tscv = TimeSeriesSplit(n_splits=3) 
model = XGBRegressor()

param_grid = {'learning_rate': [0.04],
              'nthread':[9],
    'n_estimators': [500],
    'subsample': [0.9],
    'colsample_bytree': [0.8],
    'max_depth': [9],
    'min_child_weight': [3],
    'reg_alpha': [2],
    'gamma': [0.02]}

GridSearch=GridSearchCV(estimator=model,param_grid=param_grid,cv=tscv,n_jobs=2)
GridSearch.fit(X_train,y_train)
y_pred=GridSearch.predict(X_test)

In [171]:
# gbr = GradientBoostingRegressor(n_estimators=6000,
#                                 learning_rate=0.01,
#                                 max_depth=4,
#                                 max_features='sqrt',
#                                 min_samples_leaf=15,
#                                 min_samples_split=10,
#                                 loss='huber',
#                                 random_state=42) 

In [172]:
# score = cv_rmse(xgboost)
# print("xgboost: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['xgb'] = (score.mean(), score.std())

In [173]:
MSE_xgb=mean_squared_error(y_pred,y_test)  # 10.7063378519613

print('MSE'+str(MSE_xgb))


MSE9.854975013873139


###
learning rate 0.05 10.27 0.04 9.925   
n_es:5 9.85   8deep 9  mini 3.   alpha 2 gama 1 0.02

In [174]:

# rf = RandomForestRegressor(n_estimators=1200,
#                           max_depth=15,
#                           min_samples_split=5,
#                           min_samples_leaf=5,
#                           max_features=None,
#                           oob_score=True,
#                           random_state=42)

In [117]:
#9.8549750

In [118]:
test['data_ymd']=test['datetime'].apply(lambda x:x.date())


In [119]:
testleft=test[['datetime','year','month','day','time','weekend','holidy','timesplit','data_ymd']]

In [120]:
weatherright=weather[['MaxTemp(℃)','MinTemp(℃)','MCloud(%)','cleaned','#hRedVisi(h)','WindDirect(degrees)','MWindSpeed(km/h)','data_ymd']]

In [121]:
tested=pd.merge(testleft,weatherright,on='data_ymd',how='left')

In [122]:
tested

,datetime,year,month,day,time,weekend,holidy,timesplit,data_ymd,MaxTemp(℃),MinTemp(℃),MCloud(%),cleaned,#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
0,2018-01-01 02:00:00,2018,1,1,2,0,1,Late_Night,2018-01-01,19.0,16.3,75,0,21,70,28.4
1,2018-01-01 05:00:00,2018,1,1,5,0,1,Early_Morning,2018-01-01,19.0,16.3,75,0,21,70,28.4
2,2018-01-01 07:00:00,2018,1,1,7,0,1,Early_Morning,2018-01-01,19.0,16.3,75,0,21,70,28.4
3,2018-01-01 08:00:00,2018,1,1,8,0,1,Morning,2018-01-01,19.0,16.3,75,0,21,70,28.4
4,2018-01-01 10:00:00,2018,1,1,10,0,1,Morning,2018-01-01,19.0,16.3,75,0,21,70,28.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2018-12-31 17:00:00,2018,12,31,17,0,0,Evening,2018-12-31,15.6,11.8,77,0,0,360,26.8
3500,2018-12-31 19:00:00,2018,12,31,19,0,0,Evening,2018-12-31,15.6,11.8,77,0,0,360,26.8
3501,2018-12-31 21:00:00,2018,12,31,21,0,0,Night,2018-12-31,15.6,11.8,77,0,0,360,26.8
3502,2018-12-31 22:00:00,2018,12,31,22,0,0,Night,2018-12-31,15.6,11.8,77,0,0,360,26.8


In [123]:
test_transformed=preprocessor.fit_transform(tested)
y_test_pred=GridSearch.predict(test_transformed)
# # print('GradientBoosting')
# # gbr_model_full_data = gbr.fit(X, y)

# print('xgboost')
# xgb_model_full_data = xgboost.fit(X, y)

In [128]:
pred=pd.DataFrame({'speed':list(y_test_pred)},columns=['speed'])
submission=pd.concat([test1.drop('date',axis=1),pred],axis=1)
submission.head()

,id,speed
0,0,47.111820
1,1,46.616264
2,2,38.433987
3,3,30.482649
4,4,41.279900


In [129]:
submission.shape

(3504, 2)

In [127]:
# submission.to_csv('20733823.csv',header=True, index=None,sep=',',mode='w')